In [19]:
import random
import re
import pandas as pd

df = pd.read_csv("/content/sumtext.txt",sep=" ")

In [21]:
import pandas as pd

# Assuming df is a DataFrame containing file paths in a column named 'file_path'
df = pd.DataFrame({'file_path': ['/content/sumtext.txt']})  # Replace 'your_file_path.txt' with the actual file path

# Iterate through each file path in the DataFrame
for file_path in df['file_path']:
    with open(file_path, 'r') as file:
        # Read the entire content of the file
        text = file.read()

    # Calculate the total length of the text
    total_length = len(text)
    print("Total length of the text in", file_path, ":", total_length)



Total length of the text in /content/sumtext.txt : 633


In [22]:
import random
import re
import pandas as pd

text = str(df)

# Function to preprocess text
def preprocess_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text

# Function to tokenize text into words
def tokenize_text(text):
    # Tokenize text into words
    words = text.split()
    return words

# Function to generate initial population
def generate_initial_population(population_size, num_words):
    population = []
    for _ in range(population_size):
        summary = [random.choice([0, 1]) for _ in range(num_words)]
        population.append(summary)
    return population

# Function to evaluate fitness of summaries
def evaluate_fitness(population, reference_text):
    fitness_scores = []
    for summary in population:
        summary_text = ' '.join([word for i, word in enumerate(reference_text) if summary[i] == 1])
        # Fitness is measured by the overlap of words between summary and original text
        overlap = len(set(summary_text.split()) & set(reference_text))
        fitness_scores.append(overlap)
    return fitness_scores

# Function for selection using roulette wheel selection
def selection(population, fitness_scores, num_parents):
    total_fitness = sum(fitness_scores)
    selected_parents = []
    for _ in range(num_parents):
        rand_value = random.uniform(0, total_fitness)
        cumulative_fitness = 0
        for i, fitness in enumerate(fitness_scores):
            cumulative_fitness += fitness
            if cumulative_fitness >= rand_value:
                selected_parents.append(population[i])
                break
    return selected_parents

# Function for crossover using single-point crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Function for mutation
def mutation(summary, mutation_rate):
    mutated_summary = []
    for bit in summary:
        if random.random() < mutation_rate:
            mutated_summary.append(1 - bit)  # Flip the bit
        else:
            mutated_summary.append(bit)
    return mutated_summary

# Main genetic algorithm loop
population_size = 10
num_generations = 5
mutation_rate = 0.1

# Preprocess text
preprocessed_text = preprocess_text(text)
reference_text = tokenize_text(preprocessed_text)
num_words = len(reference_text)

# Generate initial population
population = generate_initial_population(population_size, num_words)

# Evaluate fitness of initial population
fitness_scores = evaluate_fitness(population, reference_text)

# Main loop
for generation in range(num_generations):
    # Selection
    parents = selection(population, fitness_scores, 2)
    offspring = []
    for _ in range(population_size - 2):
        # Crossover
        child1, child2 = crossover(parents[0], parents[1])
        # Mutation
        child1 = mutation(child1, mutation_rate)
        child2 = mutation(child2, mutation_rate)
        offspring.extend([child1, child2])

    # Combine parents and offspring
    population = parents + offspring

    # Evaluate fitness of new population
    fitness_scores = evaluate_fitness(population, reference_text)

# Select the best summary
best_summary_index = fitness_scores.index(max(fitness_scores))
best_summary = population[best_summary_index]

# Generate final summary
final_summary = ' '.join([word for i, word in enumerate(reference_text) if best_summary[i] == 1])

print("Best Summary:", final_summary)
print("Length of text after summarization:", len(final_summary))


Best Summary: filepath 0 contentsumtexttxt
Length of text after summarization: 28
